In [1]:
import faiss
import json
import openai
import numpy as np
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.environ.get("OPENAI_API_KEY")

# Set your OpenAI API Key
openai.api_key = API_KEY

# Define paths
faiss_index_file = "./indexes-n-metadata/faiss_index_2.bin"
metadata_file = "./indexes-n-metadata/metadata_2.json"

# Load FAISS index
embedding_dim = 1536  # OpenAI text-embedding-ada-002 output size
index = faiss.IndexFlatL2(embedding_dim)
index = faiss.read_index(faiss_index_file)

# Load metadata
with open(metadata_file, "r", encoding="utf-8") as f:
    metadata = json.load(f)

print("FAISS index and metadata loaded successfully! ✅")

FAISS index and metadata loaded successfully! ✅


In [2]:
# Function to generate embeddings using OpenAI
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response["data"][0]["embedding"], dtype=np.float32)

# Function to search FAISS and retrieve relevant text
def search_faiss(query, top_k=5):
    query_embedding = get_embedding(query)
    query_vector = np.array([query_embedding], dtype=np.float32)

    # Perform similarity search
    distances, indices = index.search(query_vector, top_k)

    results = []
    for idx in indices[0]:
        if str(idx) in metadata:
            results.append(metadata[str(idx)]["text"])

    return results

# Function to ask OpenAI GPT with retrieved context
def ask_openai(query, top_k=5, model="gpt-4o-mini"):
    # Retrieve relevant text
    retrieved_chunks = search_faiss(query, top_k)

    if not retrieved_chunks:
        return "No relevant context found."

    # Format retrieved text
    context = "\n\n".join(retrieved_chunks)
    print("CONTEXT:", context)
    # Query OpenAI GPT with retrieved context
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant answering questions based on a book."},
            {"role": "user", "content": f"Here is some context:\n{context}\n\nAnswer my question: {query}"}
        ]
    )

    return response["choices"][0]["message"]["content"]


In [29]:
query = """
"Su uyur, düşman uyumaz."

Kitapta geçen bu atasözü aşağıdakilerden hangisini öğütlemektedir?

A) Tehlikeye karşı harekete geçmeyi

B) Kimsenin sözüne güvenmemeyi

C) Sıkıntıları görmezden gelmemeyi

D) Düşmana karşı uyanık olmayı
"""  
answer = ask_openai(query)

# Display results in Markdown format (Jupyter Notebook)
display(Markdown(f"### ❓ **Question:** {query}"))
display(Markdown(f"### 💡 **Answer:** {answer}"))

CONTEXT: SU UYUR DÜŞMAN UYUMAZ Baskin Rusların bir tuzağıydı. Bizim askerin zaaflarından fay- dalanmışlardı. Çünkü bizim ordu, Köprüköyü'nden hareket ettiği sırada, Rusların birkaç alay süvarisi ve bir süvari bataryası tarafın- dan takip edilmekteymiş. Bizimkiler bunu ya hesap edememişler a da boş vermişlerdi. Oysa atalarımız ne demiştir: "Su uyur düş- uyumaz!" Hele ki böylesi bir savaş hâlinde. ya man Rus kurmay subayları, bizim askerin Hasankale Kasabası'nın içine birer ikişer dağılacaklarını tahmin ederek bir kısım süvarisini kasaba etrafında karakol olarak bırakmış ve yoluna devam etmiş. Kurt İsmail Paşa, böyle bir tuzağa düşme ihtimaline karşı birlik- lere erlerin Hasankale'ye yaklaşmamalarını, belirlenen güzergâhtan ayrılmamalarını, güvenlik tedbirlerine dikkat göstermelerini bil- dirmesine rağmen erler izinsiz ilçeye girmiş, mola vermiş ve ateş yakmışlardı. Bu durumu tespit eden düşman süvarisi, yakında bulunan ufa- cık bir tepeye toplarını çıkararak bizim üzerimize gülleleri sa

### ❓ **Question:** 
"Su uyur, düşman uyumaz."

Kitapta geçen bu atasözü aşağıdakilerden hangisini öğütlemektedir?

A) Tehlikeye karşı harekete geçmeyi

B) Kimsenin sözüne güvenmemeyi

C) Sıkıntıları görmezden gelmemeyi

D) Düşmana karşı uyanık olmayı


### 💡 **Answer:** "Su uyur, düşman uyumaz." atasözü, düşmanın her an saldırıya geçebileceği ve bu nedenle dikkatli olunması gerektiğini vurgular. Bu bağlamda, doğru cevap:

**D) Düşmana karşı uyanık olmayı**